# Week 4 Assignment - Graph Theory Problems
## Kevin Obote - 190696

This notebook implements solutions for three graph theory problems using different algorithms:
1. Dijkstra's Algorithm for Shortest Path
2. PageRank Algorithm for Influence Analysis
3. Ford-Fulkerson Algorithm for Maximum Flow

In [1]:
# Import necessary libraries
import heapq
from collections import defaultdict, deque
import numpy as np

## Question 1: Shortest Path in a Road Network (Dijkstra's Algorithm)

In [2]:
def dijkstra(graph, start, end):
    """
    Implements Dijkstra's algorithm to find the shortest path between two nodes.
    
    Args:
        graph (dict): Adjacency list representation of the graph
        start (str): Starting node
        end (str): Target node
    
    Returns:
        tuple: (shortest distance, path)
    """
    # Initialize distances and paths
    distances = {node: float('infinity') for node in graph}
    distances[start] = 0
    paths = {node: [] for node in graph}
    paths[start] = [start]
    
    # Priority queue to store (distance, node)
    pq = [(0, start)]
    
    while pq:
        current_distance, current_node = heapq.heappop(pq)
        
        # If we've reached the end node
        if current_node == end:
            return current_distance, paths[current_node]
        
        # If we've found a longer path
        if current_distance > distances[current_node]:
            continue
            
        # Check all neighboring nodes
        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight
            
            # If we've found a shorter path
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                paths[neighbor] = paths[current_node] + [neighbor]
                heapq.heappush(pq, (distance, neighbor))
    
    return float('infinity'), []

In [3]:
# Test the Dijkstra's algorithm implementation
roads = {
    'A': {'B': 4, 'C': 2},
    'B': {'A': 4, 'C': 5, 'D': 10},
    'C': {'A': 2, 'B': 5, 'D': 3, 'E': 8},
    'D': {'B': 10, 'C': 3, 'E': 6, 'F': 2},
    'E': {'C': 8, 'D': 6, 'F': 4},
    'F': {'D': 2, 'E': 4}
}

# Find shortest path from A to F
distance, path = dijkstra(roads, 'A', 'F')

print("Shortest Path Analysis for Home Logistics:")
print(f"Shortest distance from City A to City F: {distance} km")
print(f"Path: {' -> '.join(path)}")

Shortest Path Analysis for Home Logistics:
Shortest distance from City A to City F: 7 km
Path: A -> C -> D -> F


## Question 2: Influence Analysis in a Social Network (PageRank Algorithm)

In [4]:
def pagerank(graph, damping_factor=0.85, epsilon=1e-8, max_iterations=100):
    """
    Implements PageRank algorithm to rank nodes by importance.
    
    Args:
        graph (dict): Adjacency list representation of the graph
        damping_factor (float): Damping factor for PageRank
        epsilon (float): Convergence threshold
        max_iterations (int): Maximum number of iterations
    
    Returns:
        dict: Node rankings
    """
    # Initialize PageRank scores
    n = len(graph)
    scores = {node: 1/n for node in graph}
    
    # Create reverse graph for incoming links
    incoming = defaultdict(list)
    for node, edges in graph.items():
        for target in edges:
            incoming[target].append(node)
    
    # Iterate until convergence
    for _ in range(max_iterations):
        new_scores = {}
        total_diff = 0
        
        for node in graph:
            # Random jump probability
            new_score = (1 - damping_factor) / n
            
            # Add contribution from incoming nodes
            for source in incoming[node]:
                new_score += damping_factor * scores[source] / len(graph[source])
                
            total_diff += abs(new_score - scores[node])
            new_scores[node] = new_score
        
        scores = new_scores
        
        # Check for convergence
        if total_diff < epsilon:
            break
    
    return scores

In [5]:
# Test the PageRank implementation
followers = {
    'Alice': ['Bob', 'Charlie'],
    'Bob': ['Charlie', 'David'],
    'Charlie': ['David'],
    'David': ['Alice'],
    'Eve': ['Alice', 'Charlie']
}

# Calculate PageRank scores
rankings = pagerank(followers)

print("Social Network Influence Analysis:")
print("\nPageRank scores for each user:")
for user, score in sorted(rankings.items(), key=lambda x: x[1], reverse=True):
    print(f"{user}: {score:.4f}")

most_influential = max(rankings.items(), key=lambda x: x[1])[0]
print(f"\nMost influential user: {most_influential}")

Social Network Influence Analysis:

PageRank scores for each user:
David: 0.2926
Alice: 0.2915
Charlie: 0.2320
Bob: 0.1539
Eve: 0.0300

Most influential user: David


## Question 3: Maximum Flow in a Water Distribution System (Ford-Fulkerson Algorithm)

In [6]:
def find_path(graph, source, sink, path, visited):
    """
    Uses DFS to find an augmenting path from source to sink.
    
    Args:
        graph (dict): Residual graph
        source (str): Source node
        sink (str): Sink node
        path (list): Current path
        visited (set): Visited nodes
    
    Returns:
        list: Augmenting path if found, else None
    """
    if source == sink:
        return path
    
    visited.add(source)
    
    for next_node, capacity in graph[source].items():
        if next_node not in visited and capacity > 0:
            result = find_path(graph, next_node, sink, path + [next_node], visited)
            if result is not None:
                return result
    
    return None

def ford_fulkerson(graph, source, sink):
    """
    Implements Ford-Fulkerson algorithm to find maximum flow.
    
    Args:
        graph (dict): Network with capacities
        source (str): Source node
        sink (str): Sink node
    
    Returns:
        int: Maximum flow
    """
    # Create residual graph
    residual = defaultdict(dict)
    for u in graph:
        for v, cap in graph[u].items():
            residual[u][v] = cap
            residual[v][u] = 0
    
    max_flow = 0
    
    while True:
        # Find an augmenting path
        path = find_path(residual, source, sink, [source], set())
        if path is None:
            break
            
        # Find minimum capacity in the path
        flow = float('infinity')
        for i in range(len(path)-1):
            u, v = path[i], path[i+1]
            flow = min(flow, residual[u][v])
        
        # Update residual capacities
        for i in range(len(path)-1):
            u, v = path[i], path[i+1]
            residual[u][v] -= flow
            residual[v][u] += flow
        
        max_flow += flow
    
    return max_flow

In [7]:
# Test the Ford-Fulkerson implementation
water_network = {
    'S': {'A': 16, 'B': 13},
    'A': {'B': 10, 'C': 12},
    'B': {'D': 14},
    'C': {'B': 9, 'T': 20},
    'D': {'C': 7, 'T': 4},
    'T': {}
}

# Calculate maximum flow
max_flow = ford_fulkerson(water_network, 'S', 'T')

print("Water Distribution System Analysis:")
print(f"Maximum flow from reservoir (S) to distribution center (T): {max_flow} units")

Water Distribution System Analysis:
Maximum flow from reservoir (S) to distribution center (T): 23 units
